In [ ]:
# Cell 1: Setup & Path Fix
import sys
import os
from pathlib import Path

# Xác định thư mục hiện tại và thư mục gốc (Root)
current_dir = Path.cwd()
# Nếu file này nằm trong thư mục 'notebooks', root sẽ là cha của nó (.parent)
# Nếu file nằm ngay ở thư mục gốc, bạn chỉ cần dùng: root_dir = current_dir
root_dir = current_dir if (current_dir / "src").exists() else current_dir.parent

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

print(f"✅ Project Root set to: {root_dir}")

# Bây giờ mới import các module từ src
from src import config, preprocessing, rf_classifier, evaluation, utils
from sklearn.model_selection import train_test_split
import numpy as np
import gc

# Khởi tạo experiment mới cho Baseline
exp_paths = utils.setup_experiment_folder()
utils.log_experiment_details(exp_path=exp_paths['root'])
print(f"🚀 Baseline Experiment Initialized.")

In [ ]:
# Cell 2 & 3: Sequential Loading and Splitting (Memory Optimized)
import numpy as np
import gc
from sklearn.model_selection import train_test_split

print("🚀 Starting Memory-Optimized Loading...")

# --- 1. XỬ LÝ TẬP 2017 ---
print("📥 Loading and Splitting 2017...")
X_17, y_17 = preprocessing.load_single_dataset_year('2017', binary_mode=True)
# Chỉ lấy 25 đặc trưng ngay lập tức để tiết kiệm RAM
X_17 = X_17.values[:, :25].astype('float32') 

X_17_train, X_17_test, y_17_train, y_17_test = train_test_split(
    X_17, y_17, test_size=0.2, random_state=config.SEED, stratify=y_17
)

# Xóa ngay dữ liệu thô 2017
del X_17, y_17
gc.collect()

# --- 2. XỬ LÝ TẬP 2018 ---
print("📥 Loading and Splitting 2018...")
X_18, y_18 = preprocessing.load_single_dataset_year('2018', binary_mode=True)
# Chỉ lấy 25 đặc trưng và ép kiểu về float32
X_18 = X_18.values[:, :25].astype('float32')

X_18_train, X_18_test, y_18_train, y_18_test = train_test_split(
    X_18, y_18, test_size=0.2, random_state=config.SEED, stratify=y_18
)

del X_18, y_18
gc.collect()

# --- 3. GỘP DỮ LIỆU (MIXED) ---
print("🔗 Merging into Mixed Datasets...")
X_train_mixed = np.vstack([X_17_train, X_18_train])
y_train_mixed = np.concatenate([y_17_train, y_18_train])

# Gộp tập Test tổng hợp (Dòng bị thiếu gây lỗi NameError)
X_test_all = np.vstack([X_17_test, X_18_test])
y_test_all = np.concatenate([y_17_test, y_18_test])

# Xóa các biến trung gian sau khi gộp để giải phóng RAM
del X_17_train, X_18_train
gc.collect()

print(f"✅ Mixed Train Shape: {X_train_mixed.shape}")
print(f"✅ Mixed Test All Shape: {X_test_all.shape}")

In [ ]:
# Cell 4 & 5: Scaling & Final Preparation
print("🔄 Scaling baseline data (Top 25 mRMR)...")
scaler = preprocessing.get_scaler()

# X_train_mixed lúc này đã là 25 cột từ Cell 3
X_train_baseline = scaler.fit_transform(X_train_mixed)

# Transform các tập test (đã là 25 cột từ Cell 2 & 3)
X_17_test_baseline = scaler.transform(X_17_test)
X_18_test_baseline = scaler.transform(X_18_test)
X_test_all_baseline = scaler.transform(X_test_all)

print(f"🔥 Baseline Ready. Final Shape: {X_train_baseline.shape}")

In [ ]:
# Cell 6: Train RF Baseline
print("🚀 Training RF Baseline (Mixed Training)...")
# Huấn luyện trên 25 đặc trưng mRMR thô
rf_baseline_model = rf_classifier.train_rf(X_train_baseline, y_train_mixed)
print("✅ Baseline Training completed.")

In [ ]:
# Cell 7: Final Evaluation
print("📊 Evaluating RF Baseline...")
test_scenarios = [
    (X_17_test_baseline, y_17_test, "Baseline on Unseen 2017"),
    (X_18_test_baseline, y_18_test, "Baseline on Unseen 2018"),
    (X_test_all_baseline, y_test_all, "Baseline Global Mixed")
]

for X_t, y_t, name in test_scenarios:
    print(f"\n--- EVALUATING BASELINE: {name} ---")
    evaluation.evaluate_model(
        model=rf_baseline_model, 
        X_test=X_t, 
        y_test=y_t, 
        save_dir=exp_paths["figures"], 
        dataset_name=name
    )